### Generate Dependency Graph for Numpy, Max Depth = 4

In [1]:
from pygraas import DependencyGraph, VulnerabilityGraph
g = DependencyGraph(package_name="numpy", package_url="https://github.com/numpy/numpy")
graph = g.build_graph(max_bacon=4)

Cloning into 'numpy'...
Updating files: 100% (2222/2222), done.
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.hook-numpy
dummymodule.py:140: WARNING: SKIPPING ILLEGAL MODULE_NAME: numpy.numpy._pyinstaller.tests.pyinstaller-smoke



changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
changing __main__ => _dummy_numpy.py
Graph built with 292 nodes and 1494 edges.


### Generate Vulnerability Graph based on the Dependency Graph

Vulnerability Graph marks the external packages which are Vulnerable, also add's the CVEs, advisory and 
vulnerable package versions as node attributes.

In [2]:
v = VulnerabilityGraph(g)
v.build_vulnerability_graph()

#### List external vulnerable packages based on the Safety-DB Database

In [3]:
vulnerables = v.get_vulnerables()
print(vulnerables)

['numpy', 'psutil', 'setuptools']


#### Degree of the "psutil" node 

In [4]:
v.get_degree(vulnerables[1])

{'in_degree': 10, 'out_degree': 1}

#### List of Public Nodes that have a path to vulnerable "setuptools"

In [18]:
# Show only the first 20 entries for demonstration.
v.get_vulnerable_public_nodes(vulnerables[2])[0: 20]

[{'grp - setuptools': True},
 {'email - setuptools': True},
 {'math - setuptools': True},
 {'importlib - setuptools': True},
 {'http - setuptools': True},
 {'collections - setuptools': True},
 {'bisect - setuptools': True},
 {'functools - setuptools': True},
 {'mimetypes - setuptools': True},
 {'fnmatch - setuptools': True},
 {'copy - setuptools': True},
 {'ntpath - setuptools': True},
 {'glob - setuptools': True},
 {'doctest - setuptools': True},
 {'enum - setuptools': True},
 {'codecs - setuptools': True},
 {'genericpath - setuptools': True},
 {'gc - setuptools': True},
 {'fcntl - setuptools': True},
 {'dataclasses - setuptools': True}]